Bibliotecas

In [1]:
import pandas as pd
import sqlite3
import re


import pandas as pd
from flask import Flask
from flask import request,Response,jsonify
import os
import json

# Populando Banco de Dados

In [2]:
#conectando no banco de dados
con = sqlite3.connect('estabelecimentos.db')

In [3]:
#definindo executor
cur = con.cursor()

# Criando tabelas
cur.execute('''drop table if EXISTS Estabelecimento''')
cur.execute('''drop table if EXISTS PolaridadeMedia''')
cur.execute('''drop table if EXISTS Avaliações''')
cur.execute('''drop table if EXISTS PrincipaisPalavras''')
cur.execute('''drop table if EXISTS Servico''')
cur.execute('''drop table if EXISTS FuncionamentoDias''')
cur.execute('''drop table if EXISTS FuncionamentoHorarios''')
cur.execute('''drop table if EXISTS Localizacao''')

cur.execute('''create table Estabelecimento
( 
  ID varchar not null primary key,
  Nome char,
  FaixaPreço float,
  Categoria varchar,
  Thumbnail varchar,
  Cluster int,
  QntdAvaliaçoes float,
  NotaMedia float
  )''')


cur.execute('''CREATE table PolaridadeMedia
( idEstabelecimento varchar not null primary key,
  PolaridadeTexto varchar not null,
  ResumoTexto varchar not null,
  PolaridadeValor float not null,
  FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
 )''')
 
cur.execute('''CREATE table Avaliações
( 
  ID int not null primary key,
  Texto int varchar,
  Nota float not null,
  Data date not null,
  PolaridadeTexto varchar,
  PolaridadeValor varchar,
  idEstabelecimento varchar not null,
  FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
 )''')

cur.execute('''CREATE TABLE PrincipaisPalavras
(
  ID int not null primary key,
  nomeEstabelecimento varchar not null,
  Frequencia int not null,
  Palavra varchar not null,
  FOREIGN KEY (nomeEstabelecimento) REFERENCES Estabelecimento(Nome)
)''')

cur.execute('''
    CREATE TABLE Servico
    (
    idEstabelecimento varchar not null primary key,
    drive_through int,
    delivery int,
    takeout int,
    dine_in int,
    FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
    )''')

cur.execute('''
    CREATE TABLE FuncionamentoDias
    (
    idEstabelecimento varchar not null primary key,
    sunday int,
    friday int,
    aberto_dia_util float,
    monday int,
    aberto_final_de_semana float,
    wednesday int,
    tuesday int,
    thursday int,
    saturday int,
    FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
    )''')


cur.execute('''
    CREATE TABLE FuncionamentoHorarios
    (
    idEstabelecimento varchar not null primary key,
    inicio varchar,
    fim varchar,
    madrugada float,
    manha float,
    tarde float,
    noite float,
    inicio_periodo varchar,
    fim_periodo varchar,
    FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
    )''')


cur.execute('''
    CREATE TABLE Localizacao
    (
    idEstabelecimento varchar not null primary key,
    Endereço varchar,
    ZonaSP varchar,
    Latitude float,
    Longitude float,
    FOREIGN KEY (idEstabelecimento) REFERENCES Estabelecimento(ID)
    )''')
con.commit()


In [4]:
#lendo datasets
df_estabelecimentos = pd.read_csv('Estabelecimento.csv')
df_avaliacoes = pd.read_csv('Avaliacoes.csv')
df_FuncionamentoDias = pd.read_csv('FuncionamentoDias.csv')
df_FuncionamentoHorarios = pd.read_csv('FuncionamentoHorarios.csv')
df_Localizacao = pd.read_csv('Localizacao.csv')
df_PolaridadeMedia = pd.read_csv('PolaridadeMedia.csv')
df_PrincipaisPalavras = pd.read_csv('PrincipaisPalavras.csv')
df_Servico = pd.read_csv('Servico.csv')

In [5]:
# Retirando aspas para não termos problemas ao adicionar os dados nas tabelas
dfs = [
    df_estabelecimentos,
    df_avaliacoes,
    df_FuncionamentoDias,
    df_FuncionamentoHorarios,
    df_Localizacao,
    df_PolaridadeMedia,
    df_PrincipaisPalavras,
    df_Servico
]

for df in dfs:
    for c in df.columns:
        df[c] = df[c].apply(lambda x: re.sub('\"', '', str(x)))

In [6]:
# verificando se temos registros de estabelecimentos repetidos
len(df_estabelecimentos), len(df_estabelecimentos.ID.unique())

(8684, 8684)

In [7]:
# populando banco de dados
print('Inserindo estabelecimentos...')

for index, row in df_estabelecimentos.iterrows():
    insert_estabelecimento = f'''
insert into Estabelecimento (
        Nome,ID ,Cluster, NotaMedia, QntdAvaliaçoes, FaixaPreço, 
        Categoria, Thumbnail
        )
values("{row['Nome']}",
"{row['ID']}",
"{row['Cluster']}",
"{row['NotaMedia']}",
"{row['QntdAvaliaçoes']}",
"{row['FaixaPreço']}",
"{row['Categoria']}",
"{row['Thumbnail']}"
)'''
    con.execute(insert_estabelecimento)
    
print('Inserindo avaliações...')

for index, row in df_avaliacoes.iterrows():
    insert_avaliacoes = f'''
    insert into Avaliações(
        ID, idEstabelecimento, Nota, Data, Texto, PolaridadeValor, PolaridadeTexto
        )
    values(
"{row['ID']}",
"{row['idEstabelecimento']}",
"{row['Nota']}",
"{row['Data']}",
"{row['Texto']}",
"{row['PolaridadeValor']}",
"{row['PolaridadeTexto']}"
    )'''
    con.execute(insert_avaliacoes)

print('Inserindo dados de Dias de Funcionamento...')
    
for index, row in df_FuncionamentoDias.iterrows(): 
    insert_FuncionamentoDias = f'''
    insert into FuncionamentoDias
    (idEstabelecimento, sunday , monday,  tuesday, wednesday, 
    thursday, friday, saturday, aberto_dia_util, aberto_final_de_semana)
    values(
        "{row['Estabelecimento_ID']}",
"{row['sunday']}",
"{row['monday']}",
"{row['tuesday']}",
"{row['wednesday']}",
"{row['thursday']}",
"{row['friday']}",
"{row['saturday']}",
"{row['aberto_dia_util']}",
"{row['aberto_final_de_semana']}"
    )'''
    con.execute(insert_FuncionamentoDias)

print('Inserindo dados de Horário de Funcionamento...')
    
for index, row in df_FuncionamentoHorarios.iterrows(): 
    insert_FuncionamentoHorarios = f'''
    insert into FuncionamentoHorarios
    (idEstabelecimento, inicio , fim,  madrugada, manha, 
    tarde, noite, inicio_periodo, fim_periodo)
    values(
   "{row['Estabelecimento_ID']}",
    "{row['inicio']}",
    "{row['fim']}",
    "{row['madrugada']}",
    "{row['manha']}",
    "{row['tarde']}",
    "{row['noite']}",
    "{row['inicio_periodo']}",
    "{row['fim_periodo']}"
    )'''
    con.execute(insert_FuncionamentoHorarios)


print('Inserindo dados de Localização...')
    
for index, row in df_Localizacao.iterrows(): 
    insert_Localizacao = f'''
    insert into Localizacao
    (idEstabelecimento, ZonaSP , Endereço,  Latitude, Longitude)
    values(
"{row['Estabelecimento_ID']}",
"{row['ZonaSP']}",
"{row['Endereço']}",
"{row['Latitude']}",
"{row['Longitude']}"
    )'''
    con.execute(insert_Localizacao)

print('Inserindo dados de Horário de PolaridadeMedia...')
    
for index, row in df_PolaridadeMedia.iterrows(): 
    insert_PolaridadeMedia = f'''
    insert into PolaridadeMedia
    (idEstabelecimento, PolaridadeTexto , ResumoTexto,  PolaridadeValor)
    values(
"{row['Estabelecimento_ID']}",
"{row['PolaridadeTexto']}",
"{row['PolaridadeValor']}",
"{row['ResumoTexto']}"
    )'''
    con.execute(insert_PolaridadeMedia)

print('Inserindo dados de Ocorrencia de Palavras...')
    
for index, row in df_PrincipaisPalavras.iterrows(): 
    insert_PrincipaisPalavras = f'''
    insert into PrincipaisPalavras
    (ID, nomeEstabelecimento , Palavra,  Frequencia)
    values(
"{row['ID']}",
"{row['Estabelecimento_Nome']}",
"{row['Palavra']}",
"{row['Frequencia']}"
    )'''
    con.execute(insert_PrincipaisPalavras)

print('Inserindo dados de Serviço...')
    
for index, row in df_Servico.iterrows(): 
    insert_Servico = f'''
    insert into Servico
    (idEstabelecimento, dine_in, takeout,  delivery, drive_through)
    values(
"{row['Estabelecimento_ID']}",
"{row['dine_in']}",
"{row['takeout']}",
"{row['delivery']}",
"{row['drive_through']}"
    )'''
    con.execute(insert_Servico)

print('\nSalvando alterações no BD...')  
print('Concluído!')
con.commit()

Inserindo estabelecimentos...
Inserindo avaliações...
Inserindo dados de Dias de Funcionamento...
Inserindo dados de Horário de Funcionamento...
Inserindo dados de Localização...
Inserindo dados de Horário de PolaridadeMedia...
Inserindo dados de Ocorrencia de Palavras...
Inserindo dados de Serviço...

Salvando alterações no BD...
Concluído!


<h1>O banco de dados foi armazenado no github de maneira pública</h1>

https://github.com/Marina-Lara/DadosEstabelecimentos.git

# API

In [8]:
# Consumindo banco de dados e trasnformando tabelas em Pandas Dataframes
strSQL_Estabelecimento ="SELECT * from  Estabelecimento"
strSQL_Avaliacoes ="SELECT * from  Avaliações"
strSQL_FuncionamentoDias ="SELECT * from  FuncionamentoDias"
strSQL_FuncionamentoHorarios ="SELECT * from  FuncionamentoHorarios"
strSQL_Localizacao ="SELECT * from  Localizacao"
strSQL_PolaridadeMedia ="SELECT * from  PolaridadeMedia"
strSQL_PrincipaisPalavras ="SELECT * from  PrincipaisPalavras"
strSQL_Servico ="SELECT * from  Servico"

df = pd.read_sql(strSQL_Estabelecimento, con)
dfava = pd.read_sql(strSQL_Avaliacoes, con)
dfdias = pd.read_sql(strSQL_FuncionamentoDias, con)
dfhorario = pd.read_sql(strSQL_FuncionamentoHorarios, con)
dflocal = pd.read_sql(strSQL_Localizacao, con)
dfpolmedia = pd.read_sql(strSQL_PolaridadeMedia, con)
dfprincipais = pd.read_sql(strSQL_PrincipaisPalavras, con)
dfserv= pd.read_sql(strSQL_Servico, con)

In [9]:
# Remove the print statement
# print(df.to_json(orient="records"))

app = Flask(__name__)

# Add a route to return the entire df as JSON
@app.route("/", methods=['GET'])
def get_csv():
    return Response(df.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/estabelecimento", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = df.loc[df['Nome'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/avaliações", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfava.loc[dfava['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/diasfuncionamento", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfdias.loc[dfdias['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/horarios", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfhorario.loc[dfhorario['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/localizacao", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dflocal.loc[dflocal['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/polaridade", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfpolmedia.loc[dfpolmedia['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/principaispalavras", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfprincipais.loc[dfprincipais['ID'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')

# Fix the function name to match the endpoint URL
@app.route("/servico", methods=['GET'])
def get_estabelecimento():
    # Convert the input parameter to string and uppercase
    cod = str(request.args.get('cod')).upper()
    prod = dfserv.loc[dfserv['idEstabelecimento'].str.upper() == cod]
    return Response(prod.to_json(orient="records"), mimetype='application/json')


if __name__ == "__main__":
    # Change the port number to avx oid conflicts
    app.run(host=os.getenv('IP', '0.0.0.0'), port=int(os.getenv('PORT', 2100)))


AssertionError: View function mapping is overwriting an existing endpoint function: get_estabelecimento